In [1]:
from selenium import webdriver

In [2]:
import requests
import time
from bs4 import BeautifulSoup
import forecastio

In [3]:
from pandas.io.json import json_normalize

In [5]:
driver = webdriver.Chrome()

### Selenium

- http://www.seleniumhq.org/
- 기본 사용법
- 파일 업로드 다운로드
    - google vision api 웹페이지에서 실습
- 스크래핑
    - gmarket 베스트 셀러 200 아이템 정보 수집
    - 네이버 메일(본인 계정) 데이터 수집
- 자동화
    - 워드프레스 블로그에 자동으로 댓글을 작성
- 이미지 스크린샷
    - youtube 페이지로 실습
    - pillow : python 이미지 처리 패키지

#### 기본 사용법

In [60]:
# 브라우저 열기
# driver - 브라우저 객체
driver = webdriver.Chrome()

In [61]:
# 페이지 이동
driver.get("http://naver.com")

In [8]:
# 브라우저의 윈도우 사이즈 조절
driver.set_window_size(800, 600) # (가로,세로)

In [10]:
# 스크롤 조절
script = "window.scrollTo(500,300)" # (가로,세로). console에서 실행해도 옮겨짐.
driver.execute_script(script)

In [42]:
# 현재 윈도우 객체를 저장
main_window = driver.current_window_handle
main_window #repr을 오버라이딩한 거

'CDwindow-E349BFEAF8BCF7E5FB8CB36FB4102B19'

In [53]:
# 새로운 윈도우 탭 열기
script = "window.open('http://google.com');"
driver.execute_script(script)

NoSuchWindowException: Message: no such window: window was already closed
  (Session info: chrome=67.0.3396.87)
  (Driver info: chromedriver=2.35.528161 (5b82f2d2aae0ca24b877009200ced9065a772e73),platform=Windows NT 6.1.7601 SP1 x86_64)


In [44]:
driver.current_window_handle # 아직 네이버임

'CDwindow-E349BFEAF8BCF7E5FB8CB36FB4102B19'

In [46]:
# 윈도우 탭 객체 확인
windows = driver.window_handles
windows
# 두 개가 나오지 않음

['CDwindow-88289FB6BE459BA8781F9760E60BA161']

In [47]:
# 현재 윈도우 탭 설정
driver.switch_to_window(windows[0]) #[1]

In [22]:
driver.switch_to_window(main_window)

In [24]:
# 페이지 새로고침
script = 'location.reload();'
driver.execute_script(script)

In [49]:
# alert
script = "alert('data science');"
driver.execute_script(script)

In [34]:
alert = driver.switch_to_alert()
alert.text

'data science'

In [50]:
# confirm - 취소 버튼과 확인 버튼이 같이 있음
script = "confirm('test selenium');"
driver.execute_script(script)

UnexpectedAlertPresentException: Alert Text: None
Message: unexpected alert open: {Alert text : data science}
  (Session info: chrome=67.0.3396.87)
  (Driver info: chromedriver=2.35.528161 (5b82f2d2aae0ca24b877009200ced9065a772e73),platform=Windows NT 6.1.7601 SP1 x86_64)


In [38]:
confirm = driver.switch_to_alert()
confirm.text

'data science'

In [51]:
confirm.dismiss() #취소 버튼 클릭
# confirm.accept() # 확인 버튼 클릭

NoAlertPresentException: Message: no alert open
  (Session info: chrome=67.0.3396.87)
  (Driver info: chromedriver=2.35.528161 (5b82f2d2aae0ca24b877009200ced9065a772e73),platform=Windows NT 6.1.7601 SP1 x86_64)


In [56]:
# input 엘리먼트에 문자열 입력하기
driver.find_element_by_css_selector("#query").send_keys("패스트 캠퍼스")

In [58]:
# 검색 버튼 클릭
driver.find_element_by_css_selector("#search_btn").click()
# element_by와 elements_by는 다른 거. 

In [59]:
# 현재 윈도우 닫기
driver.close()

In [ ]:
# 브라우저 닫기
driver.quit()
# 이렇게 닫아주는 게 좋음. selenium에서는 그냥 x버튼을 누르면 이 오브젝트가 살아있어서 메모리 차지하는 상태로 있음.
# 메모리 부족해지면 가상 메모리 사용하고 그러면 느려짐...

#### 파일 업로드
- google vision api 페이지 활용
- vision api 페이지로 이동
- iframe으로 frame을 수정
- 이미지 파일을 업로드
- 웹데이터와 도큐먼트 데이터(문자열 데이터)를 수집

In [62]:
# 구글 비전 api 페이지 이동
url = "https://cloud.google.com/vision/"
driver = webdriver.Chrome()
driver.get(url)

In [63]:
# iframe 영역으로 frame 변경
# iframe 확인할 필요 있음
iframe = driver.find_element_by_css_selector("#vision_demo_section > iframe")
driver.switch_to_frame(iframe)

In [ ]:
# 메인프레임으로 변경
# driver.switch_to_default_content() - iframe에서 frame으로 변경

In [71]:
# 파일업로드
# 파일 경로를 쓸 때는 절대경로로 사용해야 함
# \ -> /. 이거 안 하면 unicode 에러 남
file_path = "C:/Users/Administrator/Desktop/dog.png"
driver.find_element_by_css_selector("#input").send_keys(file_path)

In [74]:
# 웹데이터 가져오기
driver.find_element_by_css_selector("#webDetection").click()

In [76]:
results = driver.find_elements_by_css_selector("#card .container > .row")[:15] #elements! 주의할 것. 하나 아니라 여러 개 갖고 오니까
for result in results:
    print(
    result.find_element_by_css_selector('.name').text,
    result.find_element_by_css_selector('.score').text
    )

French Bulldog 1.29047
Boston Terrier 1.28066
Bulldog 1.27885
Bull Terrier 1.07139
Soft-coated Wheaten Terrier 0.98682
Puppy 0.8898
Border Collie 0.78608
Terrier 0.6546
Pet 0.6301
Brindle 0.5639
Dog breed 0.4977
Breed 0.4297
Veterinarian 0.4175
Black Brindle 0.4112
Dog 0


In [86]:
# 문자열 데이터 가져오기
driver.find_element_by_css_selector("#fullTextAnnotation > div").click()

In [87]:
fieldsets = driver.find_elements_by_css_selector( " .page > .container.style-scope.vs-document > fieldset")
len(fieldsets)

2

In [88]:
documents = [] # 문장 2개 들어갈 거
for fieldset in fieldsets:
    words = fieldset.find_elements_by_css_selector(" .container .word")
    print(len(words))
    word_list = []
    for word in words:
        word = word.text.replace(" ", "").replace("'", "")
        word_list.append(word)
    documents.append(" ".join(word_list))
    
documents

4
6


['In the dog house', ' 인기를 잃다 , 면목을 잃다']

In [89]:
driver.quit()

#### 스크래핑
- gmarket 베스트 200 데이터 수집
- http://corners.gmarket.co.kr/Bestsellers

In [90]:
url = 'http://corners.gmarket.co.kr/Bestsellers'
driver = webdriver.Chrome()
driver.get(url)

In [103]:
import pandas as pd

In [105]:
items = driver.find_elements_by_css_selector(".best-list:nth-child(3) > ul > li")
len(items)

dict_list = []

for item in items:
    dict_list.append({
        "title" : item.find_element_by_css_selector(".itemname").text,
        "link" : item.find_element_by_css_selector(".itemname").get_attribute("href"),
        "o-price" : item.find_element_by_css_selector(".o-price").text.replace(",","")[:-1], #'원' 없애기
        "s-price" : item.find_element_by_css_selector(".s-price").text.split(" ")[0].replace(",","")[:-1],
    })

df = pd.DataFrame(dict_list)
df

,link,o-price,s-price,title
0,http://item.gmarket.co.kr/Item?goodscode=62724...,19500,5900,더다다/원피스형 롱티/신상/루즈/박시/빅사이즈
1,http://item.gmarket.co.kr/Item?goodscode=14301...,39000,28470,[아디다스]니짜 CQ2332외 4종 택1 G/ 아디다스
2,http://item.gmarket.co.kr/Item?goodscode=63617...,30000,12900,[동원샘물미네마인]동원 샘물 2Lx9병x3팩(총 27병) /생수
3,http://item.gmarket.co.kr/Item?goodscode=41037...,24000,4900,[아망떼]1+1여름베개커버/인견/순면/30x50/40x60/50x70
4,http://item.gmarket.co.kr/Item?goodscode=14365...,39900,32900,[하기스]2018년형 에어솔솔 썸머 팬티 / 밴드 기저귀 2팩
5,http://item.gmarket.co.kr/Item?goodscode=13005...,20000,12900,[하림]하림 프로라인 치킨너겟 1kg 2봉+치킨너겟 100g(증정)
6,http://item.gmarket.co.kr/Item?goodscode=87359...,30000,13900,[동원덴마크우유]동원 인포켓치즈 20g x20개+(저키x8개 or 치즈디퍼즈)
7,http://item.gmarket.co.kr/Item?goodscode=86949...,29600,8900,유스타일팜 여름신상/빅사이즈/밴딩/데님/스커트
8,http://item.gmarket.co.kr/Item?goodscode=91202...,27100,16900,CJ 햇반210gx24개/오뚜기밥24개/컵밥10개/컵반
9,http://item.gmarket.co.kr/Item?goodscode=14298...,29000,14900,나이키 스우시 라운드 반팔 티셔츠


In [111]:
# 비어있는 o-price에 s-price 값을 대입
# broadcasting 이용해서 데이터를 true,false로 바꿔서 걔네만 =으로 해주면 됨
o_price = df["o-price"]

In [107]:
df["s-price"].head()

0     5900
1    28470
2    12900
3     4900
4    32900
Name: s-price, dtype: object

In [116]:
is_nodata = df["o-price"] == ""
df["o-price"][is_nodata]

25     
44     
89     
105    
111    
122    
137    
138    
140    
156    
172    
176    
Name: o-price, dtype: object

In [117]:
df["s-price"][is_nodata]

25     53000
44     39800
89      7000
105    39500
111    96000
122    11900
137    28900
138    27720
140    69800
156    23220
172    13900
176    32900
Name: s-price, dtype: object

In [119]:
df["o-price"][is_nodata] = df["s-price"][is_nodata]
df

,link,o-price,s-price,title
0,http://item.gmarket.co.kr/Item?goodscode=62724...,19500,5900,더다다/원피스형 롱티/신상/루즈/박시/빅사이즈
1,http://item.gmarket.co.kr/Item?goodscode=14301...,39000,28470,[아디다스]니짜 CQ2332외 4종 택1 G/ 아디다스
2,http://item.gmarket.co.kr/Item?goodscode=63617...,30000,12900,[동원샘물미네마인]동원 샘물 2Lx9병x3팩(총 27병) /생수
3,http://item.gmarket.co.kr/Item?goodscode=41037...,24000,4900,[아망떼]1+1여름베개커버/인견/순면/30x50/40x60/50x70
4,http://item.gmarket.co.kr/Item?goodscode=14365...,39900,32900,[하기스]2018년형 에어솔솔 썸머 팬티 / 밴드 기저귀 2팩
5,http://item.gmarket.co.kr/Item?goodscode=13005...,20000,12900,[하림]하림 프로라인 치킨너겟 1kg 2봉+치킨너겟 100g(증정)
6,http://item.gmarket.co.kr/Item?goodscode=87359...,30000,13900,[동원덴마크우유]동원 인포켓치즈 20g x20개+(저키x8개 or 치즈디퍼즈)
7,http://item.gmarket.co.kr/Item?goodscode=86949...,29600,8900,유스타일팜 여름신상/빅사이즈/밴딩/데님/스커트
8,http://item.gmarket.co.kr/Item?goodscode=91202...,27100,16900,CJ 햇반210gx24개/오뚜기밥24개/컵밥10개/컵반
9,http://item.gmarket.co.kr/Item?goodscode=14298...,29000,14900,나이키 스우시 라운드 반팔 티셔츠


In [127]:
# 할인율 컬럼을 추가 (e.g. 27%)
# df["s-price"] /  df["o-price"] -> type 때문에 연산이 안 됨
df["discount_rate"] = round((1 - df["s-price"].astype('int') /  df["o-price"].astype('int')) * 100,0)
df["discount_rate"] = df["discount_rate"].astype('int')
df["discount_rate"] = df["discount_rate"].astype('str')
df["discount_rate"] +=  "%"
df

,link,o-price,s-price,title,dicount_rate,discount_rate
0,http://item.gmarket.co.kr/Item?goodscode=62724...,19500,5900,더다다/원피스형 롱티/신상/루즈/박시/빅사이즈,70%,70%
1,http://item.gmarket.co.kr/Item?goodscode=14301...,39000,28470,[아디다스]니짜 CQ2332외 4종 택1 G/ 아디다스,27%,27%
2,http://item.gmarket.co.kr/Item?goodscode=63617...,30000,12900,[동원샘물미네마인]동원 샘물 2Lx9병x3팩(총 27병) /생수,57%,57%
3,http://item.gmarket.co.kr/Item?goodscode=41037...,24000,4900,[아망떼]1+1여름베개커버/인견/순면/30x50/40x60/50x70,80%,80%
4,http://item.gmarket.co.kr/Item?goodscode=14365...,39900,32900,[하기스]2018년형 에어솔솔 썸머 팬티 / 밴드 기저귀 2팩,18%,18%
5,http://item.gmarket.co.kr/Item?goodscode=13005...,20000,12900,[하림]하림 프로라인 치킨너겟 1kg 2봉+치킨너겟 100g(증정),36%,36%
6,http://item.gmarket.co.kr/Item?goodscode=87359...,30000,13900,[동원덴마크우유]동원 인포켓치즈 20g x20개+(저키x8개 or 치즈디퍼즈),54%,54%
7,http://item.gmarket.co.kr/Item?goodscode=86949...,29600,8900,유스타일팜 여름신상/빅사이즈/밴딩/데님/스커트,70%,70%
8,http://item.gmarket.co.kr/Item?goodscode=91202...,27100,16900,CJ 햇반210gx24개/오뚜기밥24개/컵밥10개/컵반,38%,38%
9,http://item.gmarket.co.kr/Item?goodscode=14298...,29000,14900,나이키 스우시 라운드 반팔 티셔츠,49%,49%


In [129]:
df = df.drop("dicount_rate", 1)
df

,link,o-price,s-price,title,discount_rate
0,http://item.gmarket.co.kr/Item?goodscode=62724...,19500,5900,더다다/원피스형 롱티/신상/루즈/박시/빅사이즈,70%
1,http://item.gmarket.co.kr/Item?goodscode=14301...,39000,28470,[아디다스]니짜 CQ2332외 4종 택1 G/ 아디다스,27%
2,http://item.gmarket.co.kr/Item?goodscode=63617...,30000,12900,[동원샘물미네마인]동원 샘물 2Lx9병x3팩(총 27병) /생수,57%
3,http://item.gmarket.co.kr/Item?goodscode=41037...,24000,4900,[아망떼]1+1여름베개커버/인견/순면/30x50/40x60/50x70,80%
4,http://item.gmarket.co.kr/Item?goodscode=14365...,39900,32900,[하기스]2018년형 에어솔솔 썸머 팬티 / 밴드 기저귀 2팩,18%
5,http://item.gmarket.co.kr/Item?goodscode=13005...,20000,12900,[하림]하림 프로라인 치킨너겟 1kg 2봉+치킨너겟 100g(증정),36%
6,http://item.gmarket.co.kr/Item?goodscode=87359...,30000,13900,[동원덴마크우유]동원 인포켓치즈 20g x20개+(저키x8개 or 치즈디퍼즈),54%
7,http://item.gmarket.co.kr/Item?goodscode=86949...,29600,8900,유스타일팜 여름신상/빅사이즈/밴딩/데님/스커트,70%
8,http://item.gmarket.co.kr/Item?goodscode=91202...,27100,16900,CJ 햇반210gx24개/오뚜기밥24개/컵밥10개/컵반,38%
9,http://item.gmarket.co.kr/Item?goodscode=14298...,29000,14900,나이키 스우시 라운드 반팔 티셔츠,49%


#### 네이버 메일 크롤링
- 네이버 페이지로 이동
- 아이디 패스워드를 입력
- 로그인 버튼을 클릭
- 메일 페이지로 이동
- 메일 페이지에서 메일 데이터를 수집

In [130]:
main_url = "http://naver.com"
mail_url = "http://mail.naver.com"

In [131]:
# 네이버 페이지로 이동
driver = webdriver.Chrome()
driver.get(main_url)

In [134]:
driver.find_element_by_css_selector("#id").send_keys("") #id 적어줘야 함
driver.find_element_by_css_selector("#pw").send_keys("") #비번 적어줘야 함

In [135]:
# 로그인 버튼 클릭
driver.find_element_by_css_selector(".btn_login > input[type=submit]").click()

In [136]:
# 메일 페이지로 이동
driver.get(mail_url)

In [138]:
# 메일 데이터 수집
mails = driver.find_elements_by_css_selector(".mailList > li")
len(mails)

15

In [ ]:
for mail in mails:
    name = mail.find_element_by_css_selector(" .name > a").text
    title = mail.find_element_by_css_selector(" .mail_title").text
    print(name, title)

In [141]:
url2 = # 주소가 깔끔하지 않으면 url decoding 해서 가져오기
driver.get(url2)

In [ ]:
mails = driver.find_elements_by_css_selector(".mailList > li")
len(mails)
for mail in mails:
    name = mail.find_element_by_css_selector(" .name > a").text
    title = mail.find_element_by_css_selector(" .mail_title").text
    print(name, title)